In [2]:
import pandas as pd
import numpy as np
import magic
from matplotlib import pyplot as plt
import importlib
%matplotlib inline
importlib.reload(magic)
import seaborn as sns

In [3]:
csv_file = 'naserator.csv'

In [4]:


data = pd.read_csv(csv_file)
data['tournament'] = data.tournament_scores/data.num_players
data['effectiveness'] = np.log(data.scores+1)/np.log(data.map_rivers_count/data.num_players)
data['density'] = np.log(data.map_rivers_count)/np.log(data.map_sites_count)
data.head()

,game_number,server_name,scores,ranking,tournament_scores,num_players,map,map_rivers_count,map_sites_count,map_mines_count,name,param1,param2,param3,tournament,effectiveness,density
0,0,Podnaserator2000Ai,81,0,3,3,Sierpinski-triangle.json,81,42,3,76ad06042d46498ab70f4d65139e7259,2.137022,1.718145,6.545865,1.000000,1.337056,1.175719
1,0,Podnaserator2000Ai,33,2,1,3,Sierpinski-triangle.json,81,42,3,8e4539b3b01d403995b92d797bf32866,1.364784,4.355336,26.758097,0.333333,1.069944,1.175719
2,0,Podnaserator2000Ai,65,1,2,3,Sierpinski-triangle.json,81,42,3,b4af4c0b2c664c4ca2a868685150eaa7,4.219238,2.002307,29.768379,0.666667,1.271196,1.175719
3,1,Podnaserator2000Ai,324,0,4,4,circle.json,65,27,4,76ad06042d46498ab70f4d65139e7259,2.137022,1.718145,6.545865,1.000000,2.074474,1.266564
4,1,Podnaserator2000Ai,94,2,2,4,circle.json,65,27,4,e0501010ef3f4986abc262769df5357a,4.340887,2.327102,12.614259,0.500000,1.633330,1.266564


In [8]:
from xgboost import XGBClassifier

In [6]:
params = ['param1','param2','param3']

test = data.loc[:  , ['game_number','ranking']+params]
test = pd.merge(left=test,right=test,left_on='game_number',right_on='game_number',how='inner')
test = test.loc[test.ranking_x!=test.ranking_y]
test['x_wins'] = np.where(test.ranking_x<test.ranking_y,1,0)

features = [p+"_"+l for p in params for l in ['x','y']]

In [9]:
model = XGBClassifier()
model.fit(test[features],test.x_wins)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [12]:
list(zip(features,model.feature_importances_))


[('param1_x', 0.17673048),
 ('param1_y', 0.17820324),
 ('param2_x', 0.12960236),
 ('param2_y', 0.15022092),
 ('param3_x', 0.18998528),
 ('param3_y', 0.17525773)]

In [13]:
from sklearn.model_selection import cross_val_score
cross_val_score(XGBClassifier(),features

SyntaxError: unexpected EOF while parsing (<ipython-input-13-9bc159903a6c>, line 2)